In [8]:
import numpy as np
import scipy as sp
import warnings
from sklearn.exceptions import DataConversionWarning
from sklearn.base import BaseEstimator, RegressorMixin

from sklearn.utils.validation import check_is_fitted
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.utils import check_X_y, check_array

In [9]:
import torch

In [10]:
print(torch.__config__.show())

PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.2 Product Build 20200624 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.6.0 (Git Hash 5ef631a030a6f73131c77892041042805a06064f)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: NO AVX
  - CUDA Runtime 11.0
  - NVCC architecture flags: -gencode;arch=compute_61,code=sm_61
  - CuDNN 8.0.2
  - Magma 2.5.2
  - Build settings: BLAS=MKL, BUILD_TYPE=Release, CXX_FLAGS= -Wno-deprecated -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -fopenmp -DNDEBUG -DUSE_FBGEMM -DUSE_QNNPACK -DUSE_PYTORCH_QNNPACK -DUSE_XNNPACK -DUSE_VULKAN_WRAPPER -O2 -fPIC -Wno-narrowing -Wall -Wextra -Werror=return-type -Wno-missing-field-initializers -Wno-type-limits -Wno-array-bounds -Wno-unknown-pragmas -Wno-sign-compare -Wno-unused-parameter -Wno-unused-variable -Wno-unused-function -Wno-unused-result -Wno-unused-local-typedefs -Wno-strict-overflow -Wno-strict-alia

In [5]:
alpha = 1e+4
L = 5000
d = 1024
bsize = 5000
N = 50000
use_original_features = True

### generate $W$

In [106]:
if use_original_features:
    W = np.random.randn(d, L+d+1)
    W[:, 1:d+1] = np.eye(d)
else:
    W = np.random.randn(d, L+1)    
    
W[:,0] = 0  # bias

W = W.astype(np.float32)

### test with single batch

In [107]:
x = np.random.rand(bsize, d) + 1
x = x.astype(np.float32)

In [108]:
%%time
h = x @ W
h[:, 0] = 1

CPU times: user 4.46 s, sys: 158 ms, total: 4.62 s
Wall time: 845 ms


In [109]:
%%time
x_gpu = torch.from_numpy(x).float().cuda()
w_gpu = torch.from_numpy(W).float().cuda()
torch.cuda.synchronize()

CPU times: user 35.3 ms, sys: 3.51 ms, total: 38.8 ms
Wall time: 9.41 ms


In [110]:
%%time
h_gpu = x_gpu.matmul(w_gpu)
h_gpu[:, 0] = 1
torch.cuda.synchronize()

CPU times: user 104 ms, sys: 4.69 ms, total: 109 ms
Wall time: 27.2 ms


In [111]:
np.allclose(h, h_gpu.cpu().numpy(), atol=1e-3)

True

### test with solver

In [122]:
x = np.random.rand(bsize, d) + 1
x = x.astype(np.float32)

y = np.random.randint(0, 3, size=(bsize, 1)) * 10
y = y.astype(np.float32)

In [123]:
%%time
h = x @ W
h[:, 0] = 1

CPU times: user 4.34 s, sys: 148 ms, total: 4.49 s
Wall time: 761 ms


In [124]:
%%time
hh = h.T @ h + np.eye(h.shape[1]) * alpha
hy = h.T @ y

CPU times: user 10.5 s, sys: 320 ms, total: 10.9 s
Wall time: 2.01 s


In [125]:
%%time
B = sp.linalg.cho_solve(sp.linalg.cho_factor(hh), hy)

CPU times: user 8.88 s, sys: 131 ms, total: 9.01 s
Wall time: 1.8 s


In [126]:
%%time
x_gpu = torch.from_numpy(x).float().cuda()
y_gpu = torch.from_numpy(y).float().cuda()
w_gpu = torch.from_numpy(W).float().cuda()
torch.cuda.synchronize()

CPU times: user 58.1 ms, sys: 487 µs, total: 58.5 ms
Wall time: 9.58 ms


In [127]:
%%time
h_gpu = x_gpu.matmul(w_gpu)
h_gpu[:, 0] = 1
torch.cuda.synchronize()

CPU times: user 141 ms, sys: 4.03 ms, total: 145 ms
Wall time: 24.1 ms


In [128]:
%%time
hh_gpu = torch.eye(W.shape[1], device="cuda") * alpha
hh_gpu += torch.mm(h_gpu.t(), h_gpu)

hy_gpu = torch.mm(h_gpu.t(), y_gpu)
torch.cuda.synchronize()

CPU times: user 223 ms, sys: 4.12 ms, total: 227 ms
Wall time: 127 ms


In [129]:
%%time
u_gpu = torch.cholesky(hh_gpu)
b_gpu = torch.cholesky_solve(hy_gpu, u_gpu)
torch.cuda.synchronize()

CPU times: user 347 ms, sys: 20 ms, total: 367 ms
Wall time: 68.4 ms


In [134]:
np.allclose(B, b_gpu.cpu().numpy(), atol=1e-2)

True

#### total runtimes

In [141]:
%%time

h = x @ W
h[:, 0] = 1
hh = h.T @ h + np.eye(h.shape[1]) * alpha
hy = h.T @ y
B = sp.linalg.cho_solve(sp.linalg.cho_factor(hh), hy)

CPU times: user 22.6 s, sys: 555 ms, total: 23.2 s
Wall time: 4.28 s


In [148]:
%%time

x_gpu = torch.from_numpy(x).float().cuda()
y_gpu = torch.from_numpy(y).float().cuda()
w_gpu = torch.from_numpy(W).float().cuda()

h_gpu = x_gpu.matmul(w_gpu)
hh_gpu = torch.eye(W.shape[1], device="cuda") * alpha
hh_gpu += torch.mm(h_gpu.t(), h_gpu)

hy_gpu = torch.mm(h_gpu.t(), y_gpu)
h_gpu[:, 0] = 1

u_gpu = torch.cholesky(hh_gpu)
b_gpu = torch.cholesky_solve(hy_gpu, u_gpu)
torch.cuda.synchronize()

CPU times: user 470 ms, sys: 32.3 ms, total: 502 ms
Wall time: 218 ms


In [85]:
%%time
b2_gpu, _ = torch.solve(hy_gpu, hh_gpu)

CPU times: user 174 ms, sys: 11.6 ms, total: 185 ms
Wall time: 30.8 ms


In [86]:
%%time
u,w,vt = torch.svd(hh_gpu)
b_svd_gpu = vt.t().mm(torch.diag(w**-1)).mm(u.t()).mm(hy_gpu)

CPU times: user 2.77 s, sys: 140 ms, total: 2.91 s
Wall time: 490 ms


In [87]:
torch.diag(w**-1)

tensor([[1.7394e-10, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 7.3006e-07, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 7.5296e-07,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.7257e-02, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 1.8681e-02,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         3.1827e-02]], device='cuda:0')

In [88]:
u,w,vt = np.linalg.svd(hh)

In [89]:
w**-1

array([1.73941277e-10, 7.30046159e-07, 7.52959338e-07, ...,
       1.01914932e-01, 1.02018615e-01, 1.02233093e-01])

In [90]:
b_svd_gpu

tensor([[-0.2051],
        [ 0.0204],
        [-0.2191],
        ...,
        [ 0.2357],
        [-0.1553],
        [ 0.0789]], device='cuda:0')

In [91]:
b_gpu

tensor([[ 0.2881],
        [ 0.2666],
        [-0.6483],
        ...,
        [ 0.0094],
        [ 0.0300],
        [-0.0045]], device='cuda:0')

In [92]:
b2_gpu

tensor([[ 0.2924],
        [ 0.2421],
        [-0.6439],
        ...,
        [ 0.0103],
        [ 0.0285],
        [-0.0028]], device='cuda:0')

In [93]:
B

array([[ 0.29529522],
       [ 0.20233413],
       [-0.66505539],
       ...,
       [ 0.00827873],
       [ 0.03210935],
       [-0.0048997 ]])

In [94]:
np.allclose(B, b_svd_gpu.cpu().numpy(), atol=1e-3)

False

In [97]:
np.abs(B - b_svd_gpu.cpu().numpy()).mean()

0.2973300394470932

In [95]:
np.abs(B - b_gpu.cpu().numpy()).mean()

0.016163080938343684

In [96]:
np.abs(B - b2_gpu.cpu().numpy()).mean()

0.01516888651084861

## test async HPELM 

In [6]:
!ls /home/akusok/HDD2TB/

00_Inception21k					MCYTD_overlap25p_n200.pkl
GPDSS10000					MCYTD_overlap50p_n200.pkl
Inception21k_feature_extractor_save_parquet.py	MCYTD_overlap90p_n200.pkl
jpg2png.py					results.err
lost+found					results_gpds.err
MCYTD_10p_n100.pkl				results_gpds.out
MCYTD_10p_n100-predict.npy			results.out
MCYTD_10p_n150.pkl				temp1.pkl
MCYTD_90p_n100.pkl				test.py
MCYTDB						untitled.txt
MCYTD_overlap10p_n200.pkl


In [18]:
import tables

In [40]:
from hpelm import HPELM
from hpelm import nnets
from hpelm.modules import make_hdf5

In [34]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [22]:
for i in range(10):
    x = np.random.randn(100_000, 100).astype(np.float32)
    y = np.random.rand(100_000, 3).astype(np.float32)
    make_hdf5(x, "/home/akusok/HDD2TB/test_x_{}.h5".format(i), dtype=np.float32)
    make_hdf5(y, "/home/akusok/HDD2TB/test_y_{}.h5".format(i), dtype=np.float32)

In [50]:
model = HPELM(inputs=100, outputs=3, accelerator="GPU", precision=np.float32, batch=2000)
model.add_neurons(10000, 'tanh')

Using CUDA GPU acceleration with Scikit-CUDA


In [51]:
i = 0
%lprun -f model.nnet.add_batch -f model.nnet._project model.add_data("/home/akusok/HDD2TB/test_x_{}.h5".format(i), "/home/akusok/HDD2TB/test_y_{}.h5".format(i))

processing batch 29/50, eta 0:00:03


Timer unit: 1e-06 s

Total time: 3.24171 s
File: /home/akusok/miniconda3/envs/torch/lib/python3.8/site-packages/hpelm/nnets/slfn_skcuda.py
Function: _project at line 170

Line #      Hits         Time  Per Hit   % Time  Line Contents
   170                                               def _project(self, X, dev=False):
   171                                                   """Projects X to H, an auxiliary function that implements a particular projection.
   172                                           
   173                                                   For actual projection, use `ELM.project()` instead.
   174                                           
   175                                                   Args:
   176                                                       X (matrix): an input data matrix, size (N * `inputs`)
   177                                                       dev (bool, optional): whether leave result in the GPU memory
   178                        

In [53]:
i = 0
%lprun -f model.nnet.add_batch -f model.nnet._project model.add_data_async("/home/akusok/HDD2TB/test_x_{}.h5".format(i), "/home/akusok/HDD2TB/test_y_{}.h5".format(i))

Timer unit: 1e-06 s

Total time: 0.610306 s
File: /home/akusok/miniconda3/envs/torch/lib/python3.8/site-packages/hpelm/nnets/slfn_skcuda.py
Function: _project at line 170

Line #      Hits         Time  Per Hit   % Time  Line Contents
   170                                               def _project(self, X, dev=False):
   171                                                   """Projects X to H, an auxiliary function that implements a particular projection.
   172                                           
   173                                                   For actual projection, use `ELM.project()` instead.
   174                                           
   175                                                   Args:
   176                                                       X (matrix): an input data matrix, size (N * `inputs`)
   177                                                       dev (bool, optional): whether leave result in the GPU memory
   178                       

In [54]:
%%time

for i in range(10):
    model.add_data_async("/home/akusok/HDD2TB/test_x_{}.h5".format(i), "/home/akusok/HDD2TB/test_y_{}.h5".format(i))

CPU times: user 33.1 s, sys: 2.36 s, total: 35.5 s
Wall time: 36.4 s


In [55]:
%%time

for i in range(10):
    model.add_data("/home/akusok/HDD2TB/test_x_{}.h5".format(i), "/home/akusok/HDD2TB/test_y_{}.h5".format(i))

CPU times: user 32.5 s, sys: 2.44 s, total: 34.9 s
Wall time: 35.8 s


In [4]:
class BatchCholeskySolver(BaseEstimator, RegressorMixin):
    
    def __init__(self, alpha=1e-7):
        self.alpha = alpha

    def _init_XY(self, X, y):
        """Initialize covariance matrices, including a separate bias term.
        """
        d_in = X.shape[1]
        self._XtX = np.eye(d_in + 1) * self.alpha
        self._XtX[0, 0] = 0
        if len(y.shape) == 1:
            self._XtY = np.zeros((d_in + 1,)) 
        else:
            self._XtY = np.zeros((d_in + 1, y.shape[1]))

    @property
    def XtY_(self):
        return self._XtY

    @property
    def XtX_(self):
        return self._XtX

    @XtY_.setter
    def XtY_(self, value):
        self._XtY = value

    @XtX_.setter
    def XtX_(self, value):
        self._XtX = value

    def _solve(self):
        """Second stage of solution (X'X)B = X'Y using Cholesky decomposition.

        Sets `is_fitted_` to True.
        """
        #todo: auto increase 'alpha' parameter if the solution fails
        B = sp.linalg.solve(self._XtX, self._XtY, assume_a='pos', overwrite_a=False, overwrite_b=False)
        self.coef_ = B[1:]
        self.intercept_ = B[0]
        self.is_fitted_ = True

    def _reset(self):
        """Erase solution and data matrices.
        """
        [delattr(self, attr) for attr in ('_XtX', '_XtY', 'coef_', 'intercept_', 'is_fitted_') if hasattr(self, attr)]

    def fit(self, X, y):
        """Solves an L2-regularized linear system like Ridge regression, overwrites any previous solutions.
        """
        self._reset()  # remove old solution
        self.partial_fit(X, y, compute_output_weights=True)
        return self
    
    def partial_fit(self, X, y, compute_output_weights=True):
        """Update model with a new batch of data.
        
        Output weight computation can be temporary turned off for faster processing. This will mark model as
        not fit. Enable `compute_output_weights` in the final call to `partial_fit`.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape=[n_samples, n_features]
            Training input samples

        y : array-like, shape=[n_samples, n_targets]
            Training targets

        compute_output_weights : boolean, optional, default True
            Whether to compute new output weights (coef_, intercept_). Disable this in intermediate `partial_fit`
            steps to run computations faster, then enable in the last call to compute the new solution.

            .. Note::
                Solution can be updated without extra data by setting `X=None` and `y=None`.
        """
        if self.alpha < 0:
            raise ValueError("Regularization parameter alpha must be non-negative.")

        # solution only
        if X is None and y is None and compute_output_weights:
            self._solve()
            return self

        # validate parameters
        X, y = check_X_y(X, y, accept_sparse=True, multi_output=True, y_numeric=True, ensure_2d=True)
        if len(y.shape) > 1 and y.shape[1] == 1:
            msg = "A column-vector y was passed when a 1d array was expected.\
                   Please change the shape of y to (n_samples, ), for example using ravel()."
            warnings.warn(msg, DataConversionWarning)
        
        # init temporary data storage
        if not hasattr(self, '_XtX'):
            self._init_XY(X, y)
        else:
            if X.shape[1] + 1 != self._XtX.shape[0]:
                n_new, n_old = X.shape[1], self._XtX.shape[0] - 1
                raise ValueError("Number of features %d does not match previous data %d." % (n_new, n_old))
                
        # compute temporary data
        X_sum = safe_sparse_dot(X.T, np.ones((X.shape[0],)))
        y_sum = safe_sparse_dot(y.T, np.ones((y.shape[0],)))
        self._XtX[0, 0] += X.shape[0]
        self._XtX[1:, 0] += X_sum
        self._XtX[0, 1:] += X_sum
        self._XtX[1:, 1:] += X.T @ X

        self._XtY[0] += y_sum
        self._XtY[1:] += X.T @ y
        
        # solve
        if not compute_output_weights:
            # mark as not fitted
            [delattr(self, attr) for attr in ('coef_', 'intercept_', 'is_fitted_') if hasattr(self, attr)]
        else:
            self._solve()
        return self
    
    def predict(self, X):
        check_is_fitted(self, 'is_fitted_')
        X = check_array(X, accept_sparse=True)
        return safe_sparse_dot(X, self.coef_, dense_output=True) + self.intercept_